In [1]:
%%capture _ 
# ^ this just silences warnings
import torch
import mlflow
# import hiddenlayer as HL

from model.collectdata_mdsA import collect_data
from model.collectdata_poca_KDE import collect_data_poca
from model.alt_loss_A import Loss
from model.training import trainNet, select_gpu
from model.utilities import load_full_state, count_parameters, Params, save_to_mlflow

from model.autoencoder_models import UNet
from model.autoencoder_models import UNetPlusPlus
#from model.allcnn_models import ACN_2i4_8L_DenseNet_BN as DenseNet
from model.autoencoder_models import DenseNet as DenseNet

In [2]:
args = Params(
    batch_size=64,
    device = select_gpu(2),
    epochs=100,
    lr=1e-4,
    experiment_name='Top Models',
    asymmetry_parameter=12
)

1 available GPUs (initially using device 0):
  0 GeForce RTX 3090


/apps/miniconda3/envs/june2020-gpu/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
'''
train_loader = collect_data(
    '/share/lazy/sokoloff/ML-data_A/Aug14_80K_train.h5',
      '/share/lazy/sokoloff/ML-data_AA/Oct03_80K_train.h5',
#     '/share/lazy/sokoloff/ML-data_AA/Oct03_40K_train.h5',
      '/share/lazy/will/ML_mdsA/June30_2020_80k_1.h5',
     '/share/lazy/will/ML_mdsA/June30_2020_80k_3.h5',
     '/share/lazy/will/ML_mdsA/June30_2020_80k_4.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_5.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_6.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_7.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_8.h5',
#     '/share/lazy/will/ML_mdsA/June30_2020_80k_9.h5',
    batch_size=args['batch_size'],
    masking=True,
    shuffle=False,
    load_XandXsq=False,
#     device = args['device'], 
    load_xy=False)

val_loader = collect_data(
    '/share/lazy/sokoloff/ML-data_AA/Oct03_20K_val.h5',
    batch_size=args['batch_size'],
    slice=slice(256 * 39),
    masking=True, 
    shuffle=False,
    load_XandXsq=False,
    load_xy=False)
'''

events = 320000
## This is used when training with the new KDE
train_loader = collect_data_poca('/share/lazy/will/data/June30_2020_80k_1.h5',
                            '/share/lazy/will/data/June30_2020_80k_3.h5',
                            '/share/lazy/will/data/June30_2020_80k_4.h5',
                            '/share/lazy/will/data/June30_2020_80k_5.h5',
                            batch_size=args['batch_size'],
                            #device=args['device'],
                            masking=True, shuffle=True,
                            load_A_and_B=True,
                            load_xy=True,
                           ## slice = slice(0,18000)
                           )

val_loader = collect_data_poca('/share/lazy/sokoloff/ML-data_AA/20K_POCA_kernel_evts_200926.h5',
                            batch_size=args['batch_size'],
                            #device=args['device'],
                            masking=True, shuffle=True,
                            load_A_and_B=True,
                            load_xy=True,
                            ##slice = slice(18000,None)
                           )

Loading data...
Loaded /share/lazy/will/data/June30_2020_80k_1.h5 in 26.39 s
Loaded /share/lazy/will/data/June30_2020_80k_3.h5 in 26.42 s
Loaded /share/lazy/will/data/June30_2020_80k_4.h5 in 26.5 s
Loaded /share/lazy/will/data/June30_2020_80k_5.h5 in 31.33 s
Constructing 320000 event dataset took 0.5751 s


RuntimeError: [enforce fail at CPUAllocator.cpp:67] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 20480000000 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
mlflow.tracking.set_tracking_uri('file:/share/lazy/pv-finder_model_repo')
mlflow.set_experiment(args['experiment_name'])

In [ ]:
model = DenseNet().to(args['device'])
#model.to("cuda:0")
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss = Loss(epsilon=1e-5,coefficient=args['asymmetry_parameter'])

parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
#load_full_state(model, optimizer, '/share/lazy/pv-finder_model_repo/24/873839e8e86949d5ba313ab0b8dfa21b/artifacts/run_stats.pyt')

run_name = 'dense_net'

# tune kernel based on gpu
#torch.backends.cudnn.benchmark=True
train_iter = enumerate(trainNet(model, optimizer, loss, train_loader, val_loader, args['epochs'], notebook=True))
with mlflow.start_run(run_name = run_name) as run:
    mlflow.log_artifact('train2.ipynb')
    for i, result in train_iter:
        print(result.cost)
        torch.save(model, 'run_stats.pyt')
        mlflow.log_artifact('run_stats.pyt')

        save_to_mlflow({
            'Metric: Training loss':result.cost,
            'Metric: Validation loss':result.val,
            'Metric: Efficiency':result.eff_val.eff_rate,
            'Metric: False positive rate':result.eff_val.fp_rate,
            'Param: Parameters':parameters,
            'Param: Events':events,
            'Param: Asymmetry':args['asymmetry_parameter'],
            'Param: Epochs':args['epochs'],
            'Param: Learning Rate':args['lr'],
        }, step=i)